# Build the database from calendars

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from pathlib import Path
import pandas as pd

# set the folder
folder = Path('/run/user/1000/gvfs/smb-share:server=istore,share=microscopy_group_share,user=jeromeb/Admin/Usage Stats/2024')
booking_types = ['default','service', 'maintenance', 'training']
divisions = ['Unknown','Cell Biology','Neurobiology','PNAS','Structural Studies','University']
groups = pd.read_csv(folder/'groups.csv')
users = pd.read_csv(folder/'users.csv')
calendars = pd.read_csv(folder/'calendars.csv')
calendars = calendars.iloc[0:3]
# get a cookie from the curl command copied from firefox dev tools
with open('cookie.txt','r') as f:
    curl_str = f.read()
# curl_str = "curl 'https://mercury.mrc-lmb.cam.ac.uk/ical' --compressed -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:131.0) Gecko/20100101 Firefox/131.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8' -H 'Accept-Language: en-GB,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br, zstd' -H 'DNT: 1' -H 'Sec-GPC: 1' -H 'Connection: keep-alive' -H 'Referer: https://mercury.mrc-lmb.cam.ac.uk/integration/linux.html?lang=en' -H 'Cookie: SESSION_CONNECT_WEBMAIL=c3667fe143e2dc76a30611b1072880b836143257f462f6a750e324fdef164259; TOKEN_CONNECT_WEBMAIL=f00b0c6e465aedc5468903660dc514453fcd40ed976cb95f506dfc6f5b3eef14' -H 'Upgrade-Insecure-Requests: 1' -H 'Sec-Fetch-Dest: document' -H 'Sec-Fetch-Mode: navigate' -H 'Sec-Fetch-Site: same-origin' -H 'Sec-Fetch-User: ?1' -H 'Priority: u=0, i'"
# cookie = 'SESSION_CONNECT_WEBMAIL=c3667fe143e2dc76a30611b1072880b836143257f462f6a750e324fdef164259; TOKEN_CONNECT_WEBMAIL=f00b0c6e465aedc5468903660dc514453fcd40ed976cb95f506dfc6f5b3eef14'
db_file = 'bookings.db'

## Update or initialize the database.

- Load all the calendars into the database.
- Update users informations

In [ ]:
from usagestats.bookingdb import BookingDB

with BookingDB(db_file, curl_str=curl_str) as bookings:
    bookings.create_calendars_from_df(calendars)
    bookings.create_instruments()
    bookings.create_booking_types(booking_types)
    bookings.create_division(divisions)
    bookings.create_groups(groups)
    df,dup = bookings.create_events()
    # df = bookings.update_users_group_from_df(users)


spinning_disk


In [28]:
len(df)

5168

In [34]:
df[df.duplicated('guid')]

,guid,user,email,start,end,subject,duration,hours,instrument
748,1bd6da22-7442-4877-99e4-b67e14101ee3,Ana Crespillo,anac@mrc-lmb.cam.ac.uk,2022-02-23 10:01:00+00:00,2022-02-23 14:31:00+00:00,ana 3s135,0 days 04:30:00,4.5,spinning_disk
3581,51A021B1-3F5B-434A-9E92-27FC3AC85433,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2023-01-10 13:01:00+00:00,2023-01-10 17:01:00+00:00,facility tour,0 days 04:00:00,4.0,zeiss_sim
3676,E28C7718-05D9-42DB-B55A-77C964A36964,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2023-01-09 16:01:00+00:00,2023-01-09 19:01:00+00:00,facility tour,0 days 03:00:00,3.0,zeiss_sim
3935,9B4E5561-DD07-402F-BB3F-413CC0ED5B81,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2023-06-30 18:01:00+01:00,2023-07-01 20:01:00+01:00,openday,1 days 02:00:00,26.0,zeiss_sim
4314,83C7346E-5241-4DE9-AFBA-C9737AD45474,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2022-11-25 16:01:00+00:00,2022-11-26 00:01:00+00:00,maintenance,0 days 08:00:00,8.0,nikon_isim
4628,4745F794-D617-49A0-9770-1A4ECB008D3B,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2023-08-21 10:01:00+01:00,2023-08-21 19:01:00+01:00,nikon service,0 days 09:00:00,9.0,nikon_isim
4633,22E40EC1-CB08-4140-8313-2F56E52777FC,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2022-11-28 00:01:00+00:00,2022-11-28 11:01:00+00:00,maintenance,0 days 11:00:00,11.0,nikon_isim
4723,DC4CABFC-2CC1-4C64-A593-D984EF5149A9,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2022-11-26 00:01:00+00:00,2022-11-27 00:01:00+00:00,maintenance,1 days 00:00:00,24.0,nikon_isim
5010,E508421F-AACF-493E-9FE2-A6569268FB1D,Ben Sutcliffe,sutcliff@mrc-lmb.cam.ac.uk,2022-11-27 00:01:00+00:00,2022-11-28 00:01:00+00:00,maintenance,1 days 00:00:00,24.0,nikon_isim


In [35]:
dup

guid          1bd6da22-7442-4877-99e4-b67e14101ee3
user                                 Ana Crespillo
email                       anac@mrc-lmb.cam.ac.uk
start                    2022-02-23 10:01:00+00:00
end                      2022-02-23 14:31:00+00:00
                              ...                 
end                      2022-11-28 00:01:00+00:00
subject                                maintenance
duration                           1 days 00:00:00
hours                                         24.0
instrument                              nikon_isim
Length: 81, dtype: object

In [ ]:
df[df.duplicated('guid')]

In [ ]:
dup

In [ ]:

df[df['guid'].duplicated()]

In [ ]:
with BookingDB(db_file, cookie) as bookings:
    print(bookings.cookie)
    

In [105]:
from urllib import request
request.Resques

In [ ]:
from ics import Event
?Event

In [ ]:
c.split(' ').index("'Cookie: ")

In [ ]:
with BookingDB(db_file) as bookings: 
    #bookings.cursor.execute('SELECT * from events')
    #rows = bookings.cursor.fetchall()
    tmp = pd.read_sql_query("SELECT users.name as 'name', groups.name as 'group' from users LEFT JOIN groups ON users.group_id=groups.id", bookings.connection)
    #tmp = pd.read_sql_query("SELECT * from users", bookings.connection)
    #groups = pd.read_sql_query("SELECT * from groups", bookings.connection)
    
#users.join(groups,left_on='group_id',right_on='id')
#rows[:10]
tmp


In [ ]:
with BookingDB(db_file) as bookings:
    bookings.cursor.execute("""SELECT rowid FROM divisions WHERE name=='Cell Biology'""")
    r = bookings.cursor.fetchone()
    print(bookings.get_division_id("Cell Biology"))


In [ ]:
bookings.cur.execute("""SELECT rowid,name FROM booking_types WHERE name=='toto'""")
rows = bookings.cur.fetchall()
#rows[0][0]
#for row in rows:
#    print(row)
len(rows)

In [ ]:
try :
    bookings.cur.execute('''INSERT INTO booking_types (name) VALUES (?)''', ('service',))
except :
    pass



In [ ]:
bookings.create_booking_types(booking_types)


In [ ]:
del bookings